# *Notebook d'analyse et de merge*

## Sommaire

<ol>
    <li><a href="#">Import des librairies</a></li>
    <li><a href="#">Chargement des données</a></li>
    <li><a href="#">Analyse et merge des données</a></li>
</ol>

## Import des librairies

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Réglage des graphiques
plt.style.use('seaborn-whitegrid')

plt.rc('font', size=12)
plt.rc('axes', titlesize=20)
plt.rc('axes', labelsize=20)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('legend', fontsize=12)

dims_fig = (25,20)

## Chargement des données

In [3]:
customers = pd.read_csv('data/olist_customers_dataset.csv', sep=',')
geolocation = pd.read_csv('data/olist_geolocation_dataset.csv', sep=',')
order_items = pd.read_csv('data/olist_order_items_dataset.csv', sep=',')
order_payments = pd.read_csv('data/olist_order_payments_dataset.csv', sep=',')
order_reviews = pd.read_csv('data/olist_order_reviews_dataset.csv', sep=',')
orders = pd.read_csv('data/olist_orders_dataset.csv', sep=',')
products = pd.read_csv('data/olist_products_dataset.csv', sep=',')
sellers = pd.read_csv('data/olist_sellers_dataset.csv', sep=',')

## Analyse et merge des données

Voici le schéma de liaisons entre les données :

![data_schema.PNG](data/img/data_schema.PNG)

Il y a des informations sur :
- Les clients
- Les commandes par clients
- Les reviews par commandes
- Les produits des commandes
- Les vendeurs
- La géolocalisation des clients et des vendeurs

L'objectif est de tout regrouper dans un fichier unique.

Le merge sera effectué pour qu'à la fin notre fichier contienne une seule ligne par commande, on va sommer, moyenner ou récupérer qu'une seule partie de certaines informations.

In [4]:
order_payments_agg = order_payments.groupby('order_id').agg({'payment_type': 'first', 'payment_installments': 'first', 'payment_value': 'sum'})
order_reviews_agg = order_reviews.groupby('order_id').agg('mean')
geolocation_agg = geolocation.groupby('geolocation_zip_code_prefix').agg('mean')
order_items_agg = order_items.groupby('order_id').agg({'order_id': 'first', 'product_id': 'first', 'seller_id': 'first', 'shipping_limit_date': 'first', 'price': 'sum', 'freight_value': 'sum'})

In [5]:
orders_merge = orders.merge(order_payments_agg, how="left", on="order_id", validate="1:m").merge(order_reviews_agg, how="left", on="order_id", validate="m:1")

In [6]:
sellers_merge = sellers.merge(geolocation_agg, how="left", left_on= ['seller_zip_code_prefix'], right_on=['geolocation_zip_code_prefix'])
sellers_merge.rename(columns={"geolocation_lat": "seller_geolocation_lat", "geolocation_lng": "seller_geolocation_lng"}, inplace=True)

In [7]:
order_items_merge = order_items_agg.merge(products, how="left", on="product_id", validate="m:1").merge(sellers_merge, how="left", on="seller_id", validate="m:1")

In [8]:
customers_merge = customers.merge(geolocation_agg, how="left", left_on= ['customer_zip_code_prefix'], right_on=['geolocation_zip_code_prefix'])
customers_merge.rename(columns={"geolocation_lat": "customer_geolocation_lat", "geolocation_lng": "customer_geolocation_lng"}, inplace=True)

In [9]:
orders_merge = orders_merge.merge(customers_merge, how="left", on="customer_id", validate="m:1").merge(order_items_merge, how="left", on="order_id", validate="1:1")

In [10]:
orders_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99441 entries, 0 to 99440
Data columns (total 36 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       99441 non-null  object 
 1   customer_id                    99441 non-null  object 
 2   order_status                   99441 non-null  object 
 3   order_purchase_timestamp       99441 non-null  object 
 4   order_approved_at              99281 non-null  object 
 5   order_delivered_carrier_date   97658 non-null  object 
 6   order_delivered_customer_date  96476 non-null  object 
 7   order_estimated_delivery_date  99441 non-null  object 
 8   payment_type                   99440 non-null  object 
 9   payment_installments           99440 non-null  float64
 10  payment_value                  99440 non-null  float64
 11  review_score                   98673 non-null  float64
 12  customer_unique_id             99441 non-null 

In [13]:
orders_merge.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_type,payment_installments,...,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,seller_geolocation_lat,seller_geolocation_lng
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,credit_card,1.0,...,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,-23.680729,-46.444238
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,boleto,1.0,...,1.0,400.0,19.0,13.0,19.0,31570.0,belo horizonte,SP,-19.807681,-43.980427
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,credit_card,3.0,...,1.0,420.0,24.0,19.0,21.0,14840.0,guariba,SP,-21.363502,-48.229601
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,credit_card,1.0,...,3.0,450.0,30.0,10.0,20.0,31842.0,belo horizonte,MG,-19.837682,-43.924053
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,credit_card,1.0,...,4.0,250.0,51.0,15.0,15.0,8752.0,mogi das cruzes,SP,-23.543395,-46.262086


On sauvegare le fichier final à la racine du projet, afin de le réutiliser dans les notebooks suivants

In [12]:
orders_merge.to_csv('merge.csv', index=False)